In [ ]:
import pandas as pd
# Importing the data
DATA_PATH_GENRE = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_w_genres.csv'
DATA_PATH_GENRE_2 = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_by_genres.csv'
DATA_PATH_ARTIST = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_by_artist.csv'
DATA_PATH = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data.csv'
genres = pd.read_csv(DATA_PATH_GENRE)
genre2 = pd.read_csv(DATA_PATH_GENRE_2)
artist = pd.read_csv(DATA_PATH_ARTIST)
control = pd.read_csv(DATA_PATH)

In [3]:
#CLEANING DATA: REMOVING '' and [] SYMBOLS

list_of_genres = []
for i in genres['genres']:
    i = i[1:-1].replace("'", "")
    list_of_genres.append(i)
    
genres.drop(['genres'], axis=1, inplace=True)
genres = genres.assign(New_genres=list_of_genres)
genres = genres.rename(columns={'New_genres':'genre', 'artists':'artist'})

#removing empty values from 'genre' column
genres = genres[genres['genre'] != '']

In [4]:
#CLEANING "ARTISTS": REMOVING [] and ''
list_of_artists  = []
for k in control['artists']:
    k = k[1:-1].replace("'", "")
    list_of_artists.append(k)
list_of_artists   
control.drop(['artists'], axis=1, inplace=True)
control = control.assign(New_artists=list_of_artists)
control = control.rename(columns={'New_artists':'artist'})

In [5]:
# Merging 
result = pd.merge(control, genres[['artist', 'genre']], how='left', on='artist')
result = result[result['genre'].isnull() == False]

In [6]:
# Transforming my dataset
# TODO - Dates need to be coherent
# TODO - might have to make it into a function. 

result['release_date'] = pd.to_datetime(result['release_date'])

In [7]:
# TODO - Build a recommender model with machine learning. 
from surprise import Reader, Dataset, SVD 
from surprise.model_selection.validation import cross_validate

# Intantiating my reader and my data
reader = Reader(rating_scale=(0,100))
data = Dataset.load_from_df(result[['artist', 'id', 'popularity']],
                           reader)
# Intatntiating my SVD 
svd = SVD()

In [ ]:
# Running a 5-fold cross-validation 
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5,
              verbose=True)

In [16]:
# Retraining the model using the entire dataset
trainset = data.build_full_trainset()
svd.fit(trainset)

In [17]:
# Model has been trained, time to use for prediction.
titles = result.copy()

titles['Estimate_Score'] = titles['id'].apply(lambda x: svd.predict('Circles', x).est)

In [18]:
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
titles.head(10)

,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,...,name,popularity,release_date,speechiness,tempo,valence,year,artist,genre,Estimate_Score
97637,0.17700,0.880,157606,0.751,0,2tnVG71enUj33Ic2nFN6kZ,0.000064,7,0.1060,-4.258,...,Ride It,90,2019-07-26,0.0874,117.948,0.884,2019,Regard,pop edm,38.734044
152624,0.00181,0.760,248036,0.964,0,6ho0GyrWZN3mhi9zVRW7xi,0.703000,2,0.0975,-5.844,...,Losing It,78,2018-07-13,0.0577,125.003,0.643,2018,FISHER,"australian house, house",38.718488
126125,0.02170,0.755,202887,0.617,0,7na7Bk98usp84FaOJFPv3d,0.000000,9,0.0625,-5.144,...,Tattoo,89,2020-02-13,0.1820,193.896,0.745,2020,Rauw Alejandro,latin,38.694540
116709,0.12500,0.805,184255,0.562,0,4lMOQR1rRW5hJgJXOtKraL,0.000005,7,0.1480,-3.117,...,Relación,86,2020-04-02,0.1050,172.094,0.767,2020,Sech,"latin, panamanian pop, reggaeton",38.687248
107191,0.01920,0.599,193837,0.887,0,0rIAC4PXANcKmitJfoqmVm,0.000001,4,0.3000,-3.967,...,Motivation,78,2019-08-16,0.0984,170.918,0.881,2019,Normani,"dance pop, pop, post-teen pop, r&b",38.669687
126050,0.29600,0.674,178480,0.881,0,5WHTFyqSii0lmT9R21abT8,0.000003,9,0.0793,-2.853,...,Don't Call Me Up,82,2019-01-18,0.1470,98.994,0.234,2019,Mabel,"dance pop, pop, post-teen pop, tropical house,...",38.653504
87804,0.19400,0.729,183907,0.625,1,1xzBco0xcoJEDXktl7Jxrr,0.009860,4,0.2480,-5.266,...,Mo Bamba,80,2018-10-05,0.0315,146.034,0.261,2018,Sheck Wes,"rap, trap, underground hip hop, vapor trap",38.634667
126063,0.12700,0.700,182920,0.748,0,4cZ3UsiKd1kUQIaq4BFIj2,0.000000,1,0.0941,-6.044,...,No One Compares To You,77,2019-01-25,0.0434,111.939,0.631,2019,Jack & Jack,"dance pop, pop, post-teen pop, social media po...",38.625901
87915,0.58700,0.572,201084,0.406,1,2nC3QhMI9reBIOWutbU3Tj,0.000004,10,0.1020,-8.624,...,Moral of the Story,84,2019-02-14,0.0427,119.812,0.265,2019,Ashe,indie pop,38.598025
135263,0.83500,0.657,189347,0.591,0,4ggabbjjuHrG06SsYnqvEs,0.000000,10,0.2480,-3.505,...,Ya Me Enteré,75,2018-05-11,0.0297,132.041,0.687,2018,Chayín Rubio,regional mexican pop,38.597986
